# **Distribución de propiedades por segmento**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

## 2022

In [48]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "2022_puebla" in nombre

    if nombre.startswith("df_") and "alfa" in nombre and "2022_tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_jul_2022_tulancingo',
 'df_alfa_q_may_2022_tulancingo',
 'df_alfa_q_oct_2022_tulancingo']

In [49]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['categoria','precio']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [50]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
for df in dataframes_list:    
    df = df.drop(columns=['categoria'])
    # Asignar la categoría real a cada registro
    df['categoria'] = df['precio'].apply(asignar_categoria)

In [51]:
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,C2,47
1,D2,39
2,D3,47
3,E1,54
4,D1,34
5,C3,14
6,B3,14
7,B1,8
8,A2,11
9,S,27


In [52]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,47,C,2
1,39,D,2
2,47,D,3
3,54,E,1
4,34,D,1
5,14,C,3
6,14,B,3
7,8,B,1
8,11,A,2
9,27,S,1


In [53]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  324


,segmento,total_por_segmento,porcentaje_del_total
0,A,24,7.407407
1,B,29,8.950617
2,C,67,20.679012
3,D,120,37.037037
4,E,57,17.592593
5,S,27,8.333333


In [54]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Asegurar que la columna de segmentos tenga el orden deseado
resultados['segmento'] = pd.Categorical(resultados['segmento'], categories=orden_segmentos, ordered=True)

# Reordenar el DataFrame según el orden de los segmentos
resultados = resultados.sort_values('segmento', ascending=False)
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            tickangle=45# Ajusta el ángulo de los ticks en el eje Y
        ),
        legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
 
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)# Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2022', carpeta='assets/graficas')

fig.show()

In [55]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Asegurar que la columna de segmentos tenga el orden deseado
resultados['segmento'] = pd.Categorical(resultados['segmento'], categories=orden_segmentos, ordered=True)

# Reordenar el DataFrame según el orden de los segmentos
resultados = resultados.sort_values('segmento', ascending=False)
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in segmentos.values else '#A9A9A9' 
                     for i, segmento in enumerate(orden_segmentos)}

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes_totales[segmentos == segmento].sum() if segmento in segmentos.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2022', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()

## 2023

In [2]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "2023_puebla" in nombre

    if nombre.startswith("df_") and "alfa"  in nombre and "2023_tulancingo" in nombre    
    #and ( "jul_2023" in nombre or "sep_2023" in nombre )
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_jul_2023_tulancingo',
 'df_alfa_q_jun_2023_tulancingo',
 'df_alfa_q_mar_2023_tulancingo',
 'df_alfa_q_may_2023_tulancingo',
 'df_alfa_q_oct_2023_tulancingo',
 'df_alfa_q_sep_2023_tulancingo']

In [3]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'segmento':'categoria','Category':'categoria','Precio':'precio'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['categoria','precio']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [5]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
for df in dataframes_list:    
    df = df.drop(columns=['categoria'])
    # Asignar la categoría real a cada registro
    df['categoria'] = df['precio'].apply(asignar_categoria)

In [6]:
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,D3,122
1,E1,90
2,C2,28
3,D2,86
4,D1,53
5,C3,54
6,S,70
7,B2,15
8,E2,15
9,A3,14


In [7]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,122,D,3
1,90,E,1
2,28,C,2
3,86,D,2
4,53,D,1
5,54,C,3
6,70,S,1
7,15,B,2
8,15,E,2
9,14,A,3


In [8]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  627


,segmento,total_por_segmento,porcentaje_del_total
0,A,34,5.422648
1,B,46,7.336523
2,C,111,17.703349
3,D,261,41.626794
4,E,105,16.746411
5,S,70,11.164274


In [9]:
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2023', carpeta='assets/graficas')

fig.show()

In [11]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Asegurar que la columna de segmentos tenga el orden deseado
resultados['segmento'] = pd.Categorical(resultados['segmento'], categories=orden_segmentos, ordered=True)

# Reordenar el DataFrame según el orden de los segmentos
resultados = resultados.sort_values('segmento', ascending=False)
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in segmentos.values else '#A9A9A9' 
                     for i, segmento in enumerate(orden_segmentos)}

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes_totales[segmentos == segmento].sum() if segmento in segmentos.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_20233', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()

## 2024

In [65]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "2023_puebla" in nombre

    if nombre.startswith("df_alfa") and "2024_tulancingo" in nombre    
    #and ( "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre )
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_tulancingo',
 'df_alfa_agosto_2024_tulancingo',
 'df_alfa_febrero_2024_tulancingo',
 'df_alfa_julio_2024_tulancingo',
 'df_alfa_junio_2024_tulancingo',
 'df_alfa_marzo_2024_tulancingo',
 'df_alfa_mayo_2024_tulancingo']

In [66]:
df_alfa_marzo_2024_tulancingo.columns

Index(['id', 'Categoria', 'Precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Seguridad',
       'Colonia', 'CP', 'url'],
      dtype='object')

In [67]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'segmento':'categoria','Category':'categoria','Categoria':'categoria','Precio':'precio'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [68]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['categoria','precio']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [69]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
for df in dataframes_list:    
    df = df.drop(columns=['categoria'])
    # Asignar la categoría real a cada registro
    df['categoria'] = df['precio'].apply(asignar_categoria)

In [70]:
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,E2,318
1,E3,140
2,D1,67
3,,66
4,E1,69
5,B2,33
6,D2,49
7,D3,21
8,C1,28
9,C3,19


In [71]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,318,E,2
1,140,E,3
2,67,D,1
3,66,NaN,1
4,69,E,1
5,33,B,2
6,49,D,2
7,21,D,3
8,28,C,1
9,19,C,3


In [72]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  871


,segmento,total_por_segmento,porcentaje_del_total
0,A,42,4.822044
1,B,62,7.118255
2,C,61,7.003444
3,D,137,15.729047
4,E,527,60.505166
5,L,4,0.459242
6,S,38,4.362801


In [73]:
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2024', carpeta='assets/graficas')

fig.show()

In [74]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Asegurar que la columna de segmentos tenga el orden deseado
resultados['segmento'] = pd.Categorical(resultados['segmento'], categories=orden_segmentos, ordered=True)

# Reordenar el DataFrame según el orden de los segmentos
resultados = resultados.sort_values('segmento', ascending=False)
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in segmentos.values else '#A9A9A9' 
                     for i, segmento in enumerate(orden_segmentos)}

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes_totales[segmentos == segmento].sum() if segmento in segmentos.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2024', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()